# 2.3.1 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot
* Chain-Of-Thought

Auflösung ist ein zentraler Faktor für die OCR Qualität. Entsprechend hilft es, bei älteren Plänen den schriftlichen Teil spaltenweise als Bild abzuspeichern (Spalte = PDF-Seite). Maximale Auflösung => Fehlerquote minimieren. (siehe 2.1.1 bzgl. Nutzungsschablone)

In [1]:
from utils.openai import OpenAI
from utils.parser import Parser
from utils.runner import Runner

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
jarvis = OpenAI(instructions)
parser = Parser()
runner = Runner()

## Einspaltiges Layout

In [ ]:
%store -r msgbaunvo
context_all = parser.text2prompts([msgbaunvo])
tt_crop_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-TT-crop.png"
# ze_tt_crop_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-TT-bb-crop.png"

### B) Art der baulichen Nutzung

1. img_prompts

In [19]:
# B3) Textinformationen: Art der baulichen Nutzung – WITHOUT CONTEXT – IMG_PROMPTS
# Komplettes PNG
instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung".'

async def run():
    return await jarvis.extractTextFromImages(instruction, [tt_crop_path])    
await runner.async_consistency_check(run)

['Hier sind die extrahierten Informationen zum Thema "Art der baulichen Nutzung" aus dem Bebauungsplan:

1.1 Bauliche Nutzung
1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO).
    - Soweit es von Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.
1.1.2 Ausnahmen im Sinne des § 3 (3) BauNVO sind gemäß § 1 (4) BauNVO nicht Bestandteil des Bebauungsplanes und daher nicht zulässig.']
#############################################
['Die Informationen zum Thema "Art der baulichen Nutzung" aus dem Bebauungsplan sind wie folgt:

1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO).
- Teilweise ist, wenn es vom Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.

1.1.2 Ausnahmen im Sinne des § 3 (3) BauNVO sind gemäß § 1 (4) BauNVO nicht Bestandteil des Bebauungsplanes und daher nicht zulässig.

1.1.3 Zahl der Vollgeschosse entsprechend den Einzeichnungen im Plan

[['Hier sind die extrahierten Informationen zum Thema "Art der baulichen Nutzung" aus dem Bebauungsplan:\n\n1.1 Bauliche Nutzung\n1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO).\n    - Soweit es von Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.\n1.1.2 Ausnahmen im Sinne des § 3 (3) BauNVO sind gemäß § 1 (4) BauNVO nicht Bestandteil des Bebauungsplanes und daher nicht zulässig.'],
 ['Die Informationen zum Thema "Art der baulichen Nutzung" aus dem Bebauungsplan sind wie folgt:\n\n1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO).\n- Teilweise ist, wenn es vom Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.\n\n1.1.2 Ausnahmen im Sinne des § 3 (3) BauNVO sind gemäß § 1 (4) BauNVO nicht Bestandteil des Bebauungsplanes und daher nicht zulässig.\n\n1.1.3 Zahl der Vollgeschosse entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).

### C) Maß der baulichen Nutzung

1. img_prompts

In [20]:
# C3) Textinformationen: Maß der baulichen Nutzung – WITHOUT CONTEXT – IMG_PROMPTS
# Komplettes PNG
instruction = 'Extrahiere alle Informationen zum Thema "Maß der baulichen Nutzung".'

async def run():
    return await jarvis.extractTextFromImages(instruction, [tt_crop_path])    
await runner.async_consistency_check(run)

['Im Bebauungsplan sind folgende Festsetzungen zum Thema "Maß der baulichen Nutzung" enthalten:

1.1 Bauliche Nutzung
1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO).
    - Teilweise ist, wenn es vom Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.
1.1.3 Zahl der Vollgeschosse entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).']
#############################################
['Hier sind die extrahierten Informationen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan:

1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO):
- Soweit es vom Gelände und den Festsetzungen her möglich ist, ist ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.

1.1.3 Zahl der Vollgeschosse entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).

2.1 Gebäudehöhen gemessen im Schnittpunkt zwischen Hauswand, neuem Gelände und Sparrenoberkante:
- 1. Vollgeschoss 3,50 m, für ein w

[['Im Bebauungsplan sind folgende Festsetzungen zum Thema "Maß der baulichen Nutzung" enthalten:\n\n1.1 Bauliche Nutzung\n1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO).\n    - Teilweise ist, wenn es vom Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.\n1.1.3 Zahl der Vollgeschosse entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).'],
 ['Hier sind die extrahierten Informationen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan:\n\n1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO):\n- Soweit es vom Gelände und den Festsetzungen her möglich ist, ist ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.\n\n1.1.3 Zahl der Vollgeschosse entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).\n\n2.1 Gebäudehöhen gemessen im Schnittpunkt zwischen Hauswand, neuem Gelände und Sparrenoberkante:\n- 1. Vollgeschoss 3,50 m, für ein weiteres Vollgeschoss 2,75 m.']

### D) Bauweise, überbaubare Grundstücksflächen

1. img_prompts

In [22]:
# D3) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITHOUT CONTEXT – IMG_PROMPTS
# Komplettes PNG
instruction = 'Extrahiere alle Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen".'

async def run():
    return await jarvis.extractTextFromImages(instruction, [tt_crop_path])
await runner.async_consistency_check(run)

['Hier sind die extrahierten Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen" aus dem Bebauungsplan:

### Bauweise, überbaubare Grundstücksflächen

1. **Bauweise (§ 22 BauNVO)**
   - Siehe Einzeichnungen im Plan.

2. **Stellung der Gebäude (§ 9 (1) 1 b BBauG)**
   - Die Stellung der Gebäude sowie die Firstrichtung sind im Plan eingezeichnet.

3. **Nebenanlagen (§ 14 BauNVO)**
   - Nebenanlagen sind in den nicht überbaubaren Grundstücksflächen nicht zugelassen.
   - Garagen sind im Haus oder an den dafür ausgewiesenen Stellen mit einem Dach zu erstellen.
   - Freistehende Einzelgaragen sind nur in Verbindung mit einer Sichtschutzwand, einem gedeckten Einstellplatz oder einer sonstigen baulichen Einbindung in die Gesamtanlage zulässig.

4. **Oberirdische Leitungen**
   - Oberirdische Leitungen sind nicht zulässig.

5. **Aufschüttungen und Abgrabungen**
   - Aufschüttungen und Abgrabungen sind auf ein Mindestmaß zu beschränken und dem Gelände anzupassen.
   - Sie sind im

[['Hier sind die extrahierten Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen" aus dem Bebauungsplan:\n\n### Bauweise, überbaubare Grundstücksflächen\n\n1. **Bauweise (§ 22 BauNVO)**\n   - Siehe Einzeichnungen im Plan.\n\n2. **Stellung der Gebäude (§ 9 (1) 1 b BBauG)**\n   - Die Stellung der Gebäude sowie die Firstrichtung sind im Plan eingezeichnet.\n\n3. **Nebenanlagen (§ 14 BauNVO)**\n   - Nebenanlagen sind in den nicht überbaubaren Grundstücksflächen nicht zugelassen.\n   - Garagen sind im Haus oder an den dafür ausgewiesenen Stellen mit einem Dach zu erstellen.\n   - Freistehende Einzelgaragen sind nur in Verbindung mit einer Sichtschutzwand, einem gedeckten Einstellplatz oder einer sonstigen baulichen Einbindung in die Gesamtanlage zulässig.\n\n4. **Oberirdische Leitungen**\n   - Oberirdische Leitungen sind nicht zulässig.\n\n5. **Aufschüttungen und Abgrabungen**\n   - Aufschüttungen und Abgrabungen sind auf ein Mindestmaß zu beschränken und dem Gelände anzupass

### E) Alle Themen in einem Prompt

1. img_prompts

In [31]:
# E1.1) Textteil – ONE COL WITH CONTEXT
instruction = 'Extrahiere alle planungsrechtlichen Festsetzungen'

async def run():
    return await jarvis.extractTextFromImagesWithContexts(instruction, [tt_crop_path], [context_all])
await runner.async_consistency_check(run)

In [33]:
# E1.2) Textteil – ONE COL WITH CONTEXT
instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'

async def run():
    return await jarvis.extractTextFromImagesWithContexts(instruction, [tt_crop_path], [context_all])    
results = await runner.async_consistency_check(run)
msg231_l04_tt = results[0]
%store msg231_l04_tt

['### Art der baulichen Nutzung

1.1 **Bauliche Nutzung**
   - **1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)**
     - Soweit es vom Gelände und den Festsetzungen her möglich ist, ist ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.
   - **1.1.2 Ausnahmen im Sinne des § 3 (3) BauNVO**
     - Gemäß § 1 (4) BauNVO nicht Bestandteil des Bebauungsplanes und daher nicht zulässig.
   - **1.1.3 Zahl der Vollgeschosse**
     - Entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).

### Maß der baulichen Nutzung

1.1 **Bauliche Nutzung**
   - **1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)**
     - Soweit es vom Gelände und den Festsetzungen her möglich ist, ist ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.
   - **1.1.3 Zahl der Vollgeschosse**
     - Entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).

### Bauweise

1.2 **Bauweise**
   - **§ 22 BauNVO**
     - Siehe Einzeichnungen im Plan.

### Überb

In [ ]:
# E2) Textteil – ONE COL WITHOUT CONTEXT
instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksflächen.'

async def run():
    return await jarvis.extractTextFromImages(instruction, [tt_crop_path])    
await runner.async_consistency_check(run)

## Zweispaltiges Layout

In [ ]:
%store -r msgbaunvo
context_all = parser.text2prompts([msgbaunvo])
folder_path = "../data/processed/bpläne/1_alles_in_einem_dokument"
img_name = "/L22-01-TT-crop"
tt_prompts = parser.imageseries2prompts(folder_path, img_name)

### B) Art der baulichen Nutzung

1. img_prompts

In [25]:
# B3) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PNG
instruction = 'Extrahiere alle Informationen zu dem Thema "Art der baulichen Nutzung". Achte auf die Stuktur. Achte auf die Stuktur.'

def run():
    return jarvis.request([
        *tt_prompts,
        {
            "type": "text",
            "text": instruction
        }
    ])
runner.consistency_check(run)

**Art der baulichen Nutzung (§§ 1-15 BauNVO)**

1. **Baugebiete (§ 1 Abs. 3 BauNVO)**

   1.1. **Allgemeines Wohngebiet (§ 4 BauNVO)**
   
   Zulässig sind:
   1. Wohngebäude
   2. nicht störende Handwerksbetriebe
   3. Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke

1.2. **Ausschluss bestimmter Arten von allgemein zulässigen Nutzungen (§ 1 Abs. 5 und 9 BauNVO)**

   Von den Nutzungen im Sinne des § 4 Abs. 2 Nr. 2 BauNVO sind folgende bestimmte Arten bzw. bauliche oder sonstige Anlagen nicht zulässig:
   1. Die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften.

1.3. **Zulässigkeit von Ausnahmen (§ 1 Abs. 6 BauNVO)**

   Ausnahmen im Sinne des § 4 Abs. 3 Nr. 1 bis 5 sind nicht zulässig.
#############################################
### Art der baulichen Nutzung (§§ 1-15 BauNVO)

#### Baugebiete (§ 1 Abs. 3 BauNVO)

##### Allgemeines Wohngebiet (§ 4 BauNVO)
Zulässig sind:
1. Wohngebäude
2. nicht störende Handwerksbetriebe
3. 

['**Art der baulichen Nutzung (§§ 1-15 BauNVO)**\n\n1. **Baugebiete (§ 1 Abs. 3 BauNVO)**\n\n   1.1. **Allgemeines Wohngebiet (§ 4 BauNVO)**\n   \n   Zulässig sind:\n   1. Wohngebäude\n   2. nicht störende Handwerksbetriebe\n   3. Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke\n\n1.2. **Ausschluss bestimmter Arten von allgemein zulässigen Nutzungen (§ 1 Abs. 5 und 9 BauNVO)**\n\n   Von den Nutzungen im Sinne des § 4 Abs. 2 Nr. 2 BauNVO sind folgende bestimmte Arten bzw. bauliche oder sonstige Anlagen nicht zulässig:\n   1. Die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften.\n\n1.3. **Zulässigkeit von Ausnahmen (§ 1 Abs. 6 BauNVO)**\n\n   Ausnahmen im Sinne des § 4 Abs. 3 Nr. 1 bis 5 sind nicht zulässig.',
 '### Art der baulichen Nutzung (§§ 1-15 BauNVO)\n\n#### Baugebiete (§ 1 Abs. 3 BauNVO)\n\n##### Allgemeines Wohngebiet (§ 4 BauNVO)\nZulässig sind:\n1. Wohngebäude\n2. nicht störende Handwerksbetriebe\n3. Anlagen für ki

### C) Maß der baulichen Nutzung

1. img_prompts

In [26]:
# C3) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PNG
instruction = 'Extrahiere alle Informationen zu dem Thema "Maß der baulichen Nutzung". Achte auf die Stuktur.'

def run():
    return jarvis.request([
        *tt_prompts,
        {
            "type": "text",
            "text": instruction
        }
    ])
runner.consistency_check(run)

Hier sind die extrahierten Informationen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan "Lindensteig I - Erweiterung":

### Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)
- **Z**: 
- **GRZ**: 0,4
- **GF**: 0,5

Diese Angaben beziehen sich auf die Grundflächenzahl (GRZ) und die Geschossflächenzahl (GF), die die bauliche Nutzung des Grundstücks regeln.
#############################################
### Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)

#### Allgemeines Wohngebiet (WA)
- **Zahl der Vollgeschosse (Z):** I
- **Grundflächenzahl (GRZ):** 0,4
- **Geschossflächenzahl (GF):** 0,5
#############################################
### Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)

1. **Zulässige Grundflächenzahl (GRZ)**
   - **GRZ:** 0,4

2. **Zulässige Geschossflächenzahl (GFZ)**
   - **GFZ:** 0,5
#############################################


['Hier sind die extrahierten Informationen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan "Lindensteig I - Erweiterung":\n\n### Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)\n- **Z**: \n- **GRZ**: 0,4\n- **GF**: 0,5\n\nDiese Angaben beziehen sich auf die Grundflächenzahl (GRZ) und die Geschossflächenzahl (GF), die die bauliche Nutzung des Grundstücks regeln.',
 '### Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)\n\n#### Allgemeines Wohngebiet (WA)\n- **Zahl der Vollgeschosse (Z):** I\n- **Grundflächenzahl (GRZ):** 0,4\n- **Geschossflächenzahl (GF):** 0,5',
 '### Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)\n\n1. **Zulässige Grundflächenzahl (GRZ)**\n   - **GRZ:** 0,4\n\n2. **Zulässige Geschossflächenzahl (GFZ)**\n   - **GFZ:** 0,5']

### D) Bauweise, überbaubare Grundstücksflächen

1. img_prompts

In [30]:
# D3) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – IMG_PROMPTS
# Komplettes PNG
instruction = 'Extrahiere alle Informationen zu dem Thema "Bauweise, überbaubare Grundstücksflächen". Achte auf die Stuktur.'

def run():
    return jarvis.request([
        *tt_prompts,
        {
            "type": "text",
            "text": instruction
        }
    ])
runner.consistency_check(run)

### Bauweise, überbaubare Grundstücksflächen

#### 1.3 Bauweise (§ 22 BauNVO)
- **E** = offene Bauweise, nur Einzelhäuser zulässig
- **a** = abweichende Bauweise für Garagen, die giebelständig auf der Grenze stehen (siehe 1.4)

#### 1.4 Garagen und Nebenanlagen (§§ 12 und 14 BauNVO)
- Garagen und Nebenanlagen sind nur innerhalb der Baugrenzen zulässig.
- Garagen sind mit dem Hauptgebäude zu verknüpfen oder in das Dach des Wohngebäudes einzubeziehen.
- Bei giebelständiger Grenzbebauung ist als abweichende Bauweise zulässig:
  - eine Gesamthöhe von 5,0 m
  - eine mittlere Gebäudehöhe von 4,0 m
  - eine Wandfläche, gemessen über fertigem Gelände, von 28 m²
- Ansonsten gelten die Maße der LBO.

#### 2. Bauordnungsrechtliche Festsetzungen (§ 9 Abs. 4 BauGB und § 73 LBO)

##### 2.1 Dachform und Dachneigung (§ 73 Abs. 1 Nr. 1 LBO)
- **Für Wohn- und Hauptgebäude:**
  - Satteldach, Dachneigung 30° - 40°
- **Für Garagen:**
  - Satteldach, Dachneigung 30° - 40° oder Flachdach

##### 2.2 Dacheinde

['### Bauweise, überbaubare Grundstücksflächen\n\n#### 1.3 Bauweise (§ 22 BauNVO)\n- **E** = offene Bauweise, nur Einzelhäuser zulässig\n- **a** = abweichende Bauweise für Garagen, die giebelständig auf der Grenze stehen (siehe 1.4)\n\n#### 1.4 Garagen und Nebenanlagen (§§ 12 und 14 BauNVO)\n- Garagen und Nebenanlagen sind nur innerhalb der Baugrenzen zulässig.\n- Garagen sind mit dem Hauptgebäude zu verknüpfen oder in das Dach des Wohngebäudes einzubeziehen.\n- Bei giebelständiger Grenzbebauung ist als abweichende Bauweise zulässig:\n  - eine Gesamthöhe von 5,0 m\n  - eine mittlere Gebäudehöhe von 4,0 m\n  - eine Wandfläche, gemessen über fertigem Gelände, von 28 m²\n- Ansonsten gelten die Maße der LBO.\n\n#### 2. Bauordnungsrechtliche Festsetzungen (§ 9 Abs. 4 BauGB und § 73 LBO)\n\n##### 2.1 Dachform und Dachneigung (§ 73 Abs. 1 Nr. 1 LBO)\n- **Für Wohn- und Hauptgebäude:**\n  - Satteldach, Dachneigung 30° - 40°\n- **Für Garagen:**\n  - Satteldach, Dachneigung 30° - 40° oder Flachda

### E) Alle Themen in einem Prompt

1. img_prompts

In [27]:
# E1) Textteil – MULTI COL WITH CONTEXT
instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>). Achte auf die Stuktur.'

def run():
    return jarvis.requestWithContext(context_all, [
        *tt_prompts,
        {
            "type": "text",
            "text": instruction
            # Optional: Zusätzlich Dach, Garage, Stelltung und Stellplatz
        }
    ])
runner.consistency_check(run)

### Art der baulichen Nutzung:
1.1 Art der baulichen Nutzung (§§ 1-15 BauNVO)
1.1.1 Baugebiete (§ 1 Abs. 3 BauNVO)
1.1.1.1 Allgemeines Wohngebiet (§ 4 BauNVO)
- Zulässig sind:
  1. Wohngebäude
  2. nicht störende Handwerksbetriebe
  3. Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke
1.1.2 Ausschluss bestimmter Arten von allgemein zulässigen Nutzungen (§ 1 Abs. 5 und 9 BauNVO)
- Nicht zulässig:
  1. Die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften
1.1.3 Zulässigkeit von Ausnahmen (§ 1 Abs. 6 BauNVO)
- Ausnahmen im Sinne des § 4 Abs. 3 Nr. 1 bis 5 sind nicht zulässig.

### Maß der baulichen Nutzung:
1.2 Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)
- WA: I
- GRZ: 0,4
- GF: 0,5

### Bauweise, überbaubare Grundstücksfläche:
1.3 Bauweise (§ 22 BauNVO)
- E = offene Bauweise, nur Einzelhäuser zulässig
- a = abweichende Bauweise für Garagen, die giebelständig auf der Grenze stehen (siehe 1.4)
1.4 Garagen und Nebenanlagen (§§ 12

['### Art der baulichen Nutzung:\n1.1 Art der baulichen Nutzung (§§ 1-15 BauNVO)\n1.1.1 Baugebiete (§ 1 Abs. 3 BauNVO)\n1.1.1.1 Allgemeines Wohngebiet (§ 4 BauNVO)\n- Zulässig sind:\n  1. Wohngebäude\n  2. nicht störende Handwerksbetriebe\n  3. Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke\n1.1.2 Ausschluss bestimmter Arten von allgemein zulässigen Nutzungen (§ 1 Abs. 5 und 9 BauNVO)\n- Nicht zulässig:\n  1. Die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften\n1.1.3 Zulässigkeit von Ausnahmen (§ 1 Abs. 6 BauNVO)\n- Ausnahmen im Sinne des § 4 Abs. 3 Nr. 1 bis 5 sind nicht zulässig.\n\n### Maß der baulichen Nutzung:\n1.2 Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)\n- WA: I\n- GRZ: 0,4\n- GF: 0,5\n\n### Bauweise, überbaubare Grundstücksfläche:\n1.3 Bauweise (§ 22 BauNVO)\n- E = offene Bauweise, nur Einzelhäuser zulässig\n- a = abweichende Bauweise für Garagen, die giebelständig auf der Grenze stehen (siehe 1.4)\n1.4 Garag

In [ ]:
# E2) Textteil – MULTI COL WITHOUT CONTEXT
instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>). Achte auf die Stuktur.'

def run():
    return jarvis.request([
        *tt_prompts,
        {
            "type": "text",
            "text": instruction
            # Optional: Zusätzlich Dach, Garage, Stelltung und Stellplatz
        }
    ])
runner.consistency_check(run)